In [5]:
import os
import cv2
import glob
import pickle
import numpy as np
from tqdm import tqdm
from keras.utils import normalize
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from matplotlib.patches import Rectangle

SIZE_X = 128
SIZE_Y = 128
n_classes=10
parent_directory = r'E:\Segmentation\datasets\FloodNet-Supervised_v1.0'

In [2]:
def load_images(parent_directory, split='train'):
    images = []
    masks = [] 

    image_folder_path = f'{parent_directory}\{split}\{split}-org-img'
    for img_path in tqdm(glob.glob(os.path.join(image_folder_path, "*.jpg"))):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        images.append(img)

        mask_path = (img_path.replace('org', 'label')).replace('jpg', 'png')
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        masks.append(mask)
                
    images = np.array(images)
    masks = np.array(masks)

    print(f'{split.upper()}: Images loaded: {images.shape[0]}')
    print(f'{split.upper()}: Masks loaded: {masks.shape[0]}')

    return images, masks

In [4]:
def preprocess_data(images, masks, unet_preporcessing):
    if unet_preporcessing:
        images = unet_preporcessing(images)
    else:
        images = normalize(images, axis=1)
        
    masks = np.expand_dims(masks, axis=-1)
    masks = to_categorical(masks, num_classes=n_classes)
    masks = masks.reshape((masks.shape[0], masks.shape[1], masks.shape[2], n_classes))

    return images, masks

In [6]:
def display_images_with_masks(image, mask, predicted):
    
    class_map= {'Background':0, 'Building-flooded':1, 'Building-non-flooded':2, 'Road-flooded':3, 'Road-non-flooded':4, 'Water':5, 'Tree':6, 'Vehicle':7, 'Pool':8, 'Grass':9}

    color_map = {
        "Background": [0, 0, 0],
        "Building-flooded": [255, 0, 0],
        "Building-non-flooded": [0, 255, 0],
        "Road-flooded": [0, 255, 120],
        "Road-non-flooded": [0, 0, 255],
        "Water": [255, 0, 255],
        "Tree": [70, 70, 70],
        "Vehicle": [102, 102, 156],
        "Pool": [190, 153, 153],
        "Grass": [180, 165, 180]
    }

    handles = [
        Rectangle((0, 0), 1, 1, color=np.array(c)/255) for n, c in color_map.items()
    ]
    labels = [n for n, c in color_map.items()]

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    plt.title('Image')

    plt.subplot(1, 3, 2)
    mask_colored = np.zeros_like(image, dtype=np.uint8)
    for class_name, class_idx in class_map.items():
        color = color_map[class_name]
        mask_indices = np.where(mask == class_idx)
        mask_colored[mask_indices[0], mask_indices[1], :] = color
    plt.imshow(cv2.cvtColor(mask_colored, cv2.COLOR_BGR2RGB))
    plt.title('Ground-Truth Mask')

    plt.subplot(1, 3, 3)
    mask_colored1 = np.zeros_like(image, dtype=np.uint8)
    for class_name1, class_idx1 in class_map.items():
        color1 = color_map[class_name1]
        mask_indices1 = np.where(predicted == class_idx1)
        mask_colored1[mask_indices1[0], mask_indices1[1], :] = color1
    plt.imshow(cv2.cvtColor(mask_colored1, cv2.COLOR_BGR2RGB))
    plt.title('Predicted Mask')

    plt.legend(handles, labels, bbox_to_anchor =(-0.8,-0.5), loc='lower center', ncol=5)
    plt.show()

In [7]:
#Load the images
train_images, train_masks = load_images(parent_directory, split='train')
val_images, val_masks = load_images(parent_directory, split='val')

 37%|███▋      | 538/1445 [02:32<05:01,  3.00it/s]

In [ ]:
data_to_save = {
    'train_images': train_images,
    'train_masks': train_masks,
    'val_images': val_images,
    'val_masks': val_masks
}

pickle_file_path = '/dataset/train_and_val_data.pickle'

with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(data_to_save, pickle_file)

print(f'Data saved to {pickle_file_path}')

In [ ]:
pickle_file_path = '/dataset/train_and_val_data.pickle'

with open(pickle_file_path, 'rb') as pickle_file:
    loaded_data = pickle.load(pickle_file)

train_images_loaded = loaded_data['train_images']
train_masks_loaded = loaded_data['train_masks']
val_images_loaded = loaded_data['val_images']
val_masks_loaded = loaded_data['val_masks']

print('Shapes of loaded data:')
print('Train Images:', train_images_loaded.shape)
print('Train Masks:', train_masks_loaded.shape)
print('Val Images:', val_images_loaded.shape)
print('Val Masks:', val_masks_loaded.shape)


In [ ]:
import segmentation_models as sm

BACKBONE = 'resnet34'
# preprocess_input = sm.get_preprocessing(BACKBONE)

# x_train = preprocess_input(train_images)
# x_val = preprocess_input(val_images)

model = sm.Unet(BACKBONE, input_shape=(128, 128, 3), classes=10, encoder_weights='imagenet',  activation='softmax')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)
model.summary()

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=16, epochs=100, validation_data=(x_val, y_val))

In [ ]:
model.save('UNet_model_with_100_epochs.hdf5')

In [ ]:
model.load_weights('UNet_model_with_50_epochs.hdf5')

In [ ]:
test_images, test_masks = load_images(parent_directory, split='test')
test_masks = np.expand_dims(test_masks, axis=-1)
test_masks_cat = to_categorical(test_masks, num_classes=n_classes)
test_masks_cat = test_masks_cat.reshape((test_masks.shape[0], test_masks.shape[1], test_masks.shape[2], n_classes))
test_images_seperate = test_images
test_images = normalize(test_images, axis=1)

print(f'\nImages Shape: {test_images[0].shape}')
print(f'Masks Shape: {test_masks_cat[0].shape}')

In [ ]:
_, acc = model.evaluate(test_images, test_masks_cat, verbose=0)
print("Accuracy is = ", (acc * 100.0), "%")

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from keras.metrics import MeanIoU
import numpy as np

y_pred = model.predict(test_images)
y_pred_argmax = np.argmax(y_pred, axis=3)
predicted_masks = np.argmax(y_pred, axis=-1)
predicted_masks = np.expand_dims(predicted_masks, axis=-1)

class_names = ["Background", "Building Flooded", "Building Non-Flooded", "Road Flooded", "Road Non-Flooded",
               "Water", "Tree", "Vehicle", "Pool", "Grass"]

n_classes = len(class_names)  
IOU_keras = MeanIoU(num_classes=n_classes)

IOU_keras.update_state(test_masks, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)

class_IoUs = []
for i in range(n_classes):
    class_IoU = values[i, i] / np.sum(values[i, :])
    class_IoUs.append(class_IoU)
    print(f"IoU for class {class_names[i]} is: {class_IoU}")

print("Overall mean IoU =", np.mean(class_IoUs))

In [ ]:
image_id = 13
image = test_images_seperate[image_id]
ground_truth_mask = test_masks[image_id]
predicted_mask = predicted_masks[image_id]

print(f'Image Shape: {image.shape}')
print(f'Ground Truth Mask Shape: {ground_truth_mask.shape}')
print(f'Predicted Mask Shape: {predicted_mask.shape}')

display_images_with_masks(image, ground_truth_mask, predicted_mask)